<a href="https://colab.research.google.com/github/AshbeeKim/Kaggle_task/blob/main/taxi_eda.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
from google.colab import drive  
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib
from matplotlib import pyplot as pltm
%matplotlib inline
import sklearn
# import mglearn
import graphviz
import warnings
warnings.filterwarnings('ignore')
# import statsmodels.api as sm

# 최대 줄 수 설정
pd.set_option('display.max_rows', 1500)
# 최대 열 수 설정
pd.set_option('display.max_columns', 1500)
# 표시할 가로의 길이
pd.set_option('display.width', 1500)

In [ ]:
data = pd.read_csv('/content/drive/MyDrive/Task/nyctaxitripduration/train.csv')
tstdata = pd.read_csv('/content/drive/MyDrive/Task/nyctaxitripduration/test.csv')
spSubms = pd.read_csv('/content/drive/MyDrive/Task/nyctaxitripduration/sample_submission.csv')

In [ ]:
train = data.copy()
test = tstdata.copy()
sS = spSubms.copy()

In [ ]:
print(train.info())
print(test.info())
print(sS.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1458644 entries, 0 to 1458643
Data columns (total 11 columns):
 #   Column              Non-Null Count    Dtype  
---  ------              --------------    -----  
 0   id                  1458644 non-null  object 
 1   vendor_id           1458644 non-null  int64  
 2   pickup_datetime     1458644 non-null  object 
 3   dropoff_datetime    1458644 non-null  object 
 4   passenger_count     1458644 non-null  int64  
 5   pickup_longitude    1458644 non-null  float64
 6   pickup_latitude     1458644 non-null  float64
 7   dropoff_longitude   1458644 non-null  float64
 8   dropoff_latitude    1458644 non-null  float64
 9   store_and_fwd_flag  1458644 non-null  object 
 10  trip_duration       1458644 non-null  int64  
dtypes: float64(4), int64(3), object(4)
memory usage: 122.4+ MB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 625134 entries, 0 to 625133
Data columns (total 9 columns):
 #   Column              Non-Null Count   Dty

결측치없음
object와 int, float이 섞여있는 형태

test에는 dropoff_datatime이 없음

dropoff_datetime을 맞춰야 하나? 우선은 train의 dropoff_datatime 버리고, duration을 예측해서 dropoff_datetime을 구해야 하나?

In [ ]:
train.pickup_datetime = pd.to_datetime(train.pickup_datetime)
train.dropoff_datetime = pd.to_datetime(train.dropoff_datetime)
train.trip_duration = pd.to_timedelta(train.trip_duration,unit='sec')

# import time
# train.dropoff_datetime[0]-train.pickup_datetime[0]

In [ ]:
print(train.shape)
print(test.shape)
print(sS.shape)

(1458644, 11)
(625134, 9)
(625134, 2)


In [ ]:
for rows in train.iloc[:]:
  print(rows)

id
vendor_id
pickup_datetime
dropoff_datetime
passenger_count
pickup_longitude
pickup_latitude
dropoff_longitude
dropoff_latitude
store_and_fwd_flag
trip_duration


In [ ]:
n=0
for do, pu, dt in zip(train.dropoff_datetime, train.pickup_datetime, train.trip_duration):
  print(do, type(do))
  print(pu, type(pu))
  print(dt, type(dt))
  # if do-pu==dt:
    # n+=1
# print(n)    

Streaming output truncated to the last 5000 lines.
2016-01-01 01:50:09 <class 'str'>
690 <class 'int'>
2016-01-06 15:26:50 <class 'str'>
2016-01-06 15:12:36 <class 'str'>
854 <class 'int'>
2016-02-25 07:49:00 <class 'str'>
2016-02-25 07:30:20 <class 'str'>
1120 <class 'int'>
2016-06-20 19:04:28 <class 'str'>
2016-06-20 19:02:31 <class 'str'>
117 <class 'int'>
2016-05-10 13:06:03 <class 'str'>
2016-05-10 12:58:15 <class 'str'>
468 <class 'int'>
2016-02-13 03:43:07 <class 'str'>
2016-02-13 03:36:58 <class 'str'>
369 <class 'int'>
2016-03-27 20:19:48 <class 'str'>
2016-03-27 20:12:15 <class 'str'>
453 <class 'int'>
2016-05-19 20:56:49 <class 'str'>
2016-05-19 20:51:52 <class 'str'>
297 <class 'int'>
2016-04-18 20:27:50 <class 'str'>
2016-04-18 20:24:58 <class 'str'>
172 <class 'int'>
2016-04-11 21:25:18 <class 'str'>
2016-04-11 21:17:42 <class 'str'>
456 <class 'int'>
2016-05-02 22:21:09 <class 'str'>
2016-05-02 22:17:52 <class 'str'>
197 <class 'int'>
2016-06-16 10:03:01 <class 'str'>
20

KeyboardInterrupt: ignored

In [ ]:
print(train.columns)
print(test.columns)
print(sS.columns)

Index(['id', 'vendor_id', 'pickup_datetime', 'dropoff_datetime', 'passenger_count', 'pickup_longitude', 'pickup_latitude', 'dropoff_longitude', 'dropoff_latitude', 'store_and_fwd_flag', 'trip_duration'], dtype='object')
Index(['id', 'vendor_id', 'pickup_datetime', 'passenger_count', 'pickup_longitude', 'pickup_latitude', 'dropoff_longitude', 'dropoff_latitude', 'store_and_fwd_flag'], dtype='object')
Index(['id', 'trip_duration'], dtype='object')


In [ ]:
test = pd.concat([test, sS['trip_duration']], join='outer', axis=1)

In [ ]:
n=0
for i in train.columns:
  n+=1
print(n)

n=0
for i in test.columns:
  n+=1
print(n)

11
10


In [ ]:
#['id', 'vendor_id', 'pickup_datetime', 'dropoff_datetime', 'passenger_count', 'pickup_longitude', 'pickup_latitude', 'dropoff_longitude', 'dropoff_latitude', 'store_and_fwd_flag', 'trip_duration']
all_col_trn = train.columns
all_col_tst = test.columns


for trn_unq, tst_unq in zip(all_col_trn, all_col_tst):
  ntrn = 0
  ntst = 0
  print(f'train {trn_unq} : {train[trn_unq].unique()}')
  for i in train[trn_unq].unique():
    ntrn += 1
  print('-'*250)
  print(f'test {tst_unq} : {test[tst_unq].unique()}')
  for j in test[tst_unq].unique():
    ntst += 1
  print('-'*250)
  print(f'train {trn_unq} : {ntrn}개')
  print(f'test {tst_unq} : {ntst}개')
  print('='*250)

train id : ['id2875421' 'id2377394' 'id3858529' ... 'id2304944' 'id2714485'
 'id1209952']
----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
test id : ['id3004672' 'id3505355' 'id1217141' ... 'id2568735' 'id1384355'
 'id0621643']
----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
train id : 1458644개
test id : 625134개
train vendor_id : [2 1]
----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
test v

In [ ]:
for trn_unq, tst_unq in zip(all_col_trn, all_col_tst):
  ntrn = 0
  ntst = 0
  for i in train[trn_unq].unique():
    ntrn += 1
  for j in test[tst_unq].unique():
    ntst += 1
  print(f'train {trn_unq} : {ntrn}개')
  print(f'test {tst_unq} : {ntst}개')
  print('='*250)

train id : 1458644개
test id : 625134개
train vendor_id : 2개
test vendor_id : 2개
train pickup_datetime : 1380222개
test pickup_datetime : 610581개
train dropoff_datetime : 1380377개
test passenger_count : 8개
train passenger_count : 10개
test pickup_longitude : 18926개
train pickup_longitude : 23047개
test pickup_latitude : 38408개
train pickup_latitude : 45245개
test dropoff_longitude : 26476개
train dropoff_longitude : 33821개
test dropoff_latitude : 51439개
train dropoff_latitude : 62519개
test store_and_fwd_flag : 2개
train store_and_fwd_flag : 2개
test trip_duration : 1개
